<div style="background-image: linear-gradient(145deg, rgba(35, 47, 62, 1) 0%, rgba(0, 49, 129, 1) 40%, rgba(32, 116, 213, 1) 60%, rgba(244, 110, 197, 1) 85%, rgba(255, 173, 151, 1) 100%); padding: 1rem 2rem; width: 95%"><img style="width: 60%;" src="../../images/MLU_logo.png"></div>

# <a name="0">MLU Mathematical Fundamentals for Machine Learning</a>
# <a name="0">Lecture 2: Advanced linear algebra</a>
## <a name="0">Lab 2.2: Dimensionality reduction and Principal Component Regression</a>

 1. <a href="#1">Principal Component Analysis</a> 
 2. <a href="#2">Visualization of the Fashion MNIST dataset</a> 
 3. <a href="#3">Principal Component Regression</a> 
 
**Principal Component Analysis (PCA)** is a popular technique for dimensionality reduction. In many datasets, there can be a lot of features (or variables), and some of them may be redundant or only add noise. PCA helps simplify these datasets by transforming them into a smaller set of new features, called principal components, which capture the most important information (variance) from the original data.

In this lab you will explore how PCA can be leveraged to visualize high-dimensional datasets and how to combine principal components with linear regression to build models with less features in what's called Principal Component Regression.

In [ ]:
# Upgrade libraries
!pip install -q --upgrade pip
!pip install -q --upgrade scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml, load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score

from IPython.display import Markdown, display

%matplotlib inline

## <a name="1">1. Principal Component Analysis</a>
(<a href="#0">Go to top</a>)

Principal component analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components. To do that, it leverages the eigendecomposition of symmetric matrices that you've seen in the Lecture 2 slides. 

The transformation into principal components is defined in such a way that the first principal component has the largest possible variance. This means that it accounts for as much of the variability in the data as possible, and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal the preceding components. The resulting vectors are an uncorrelated orthogonal basis set.

PCA is sensitive to the relative scaling of the original variables.

#### Steps to compute the PCA

For a dataset represented by a matrix $X$, where each row is an observation and each column is a feature:

 - Center the Data: Subtract the mean of each feature so that the dataset has a mean of zero.
 - Compute Covariance Matrix: Calculate the covariance matrix of the data.
 - Calculate Eigenvalues and Eigenvectors: Compute the eigenvalues and eigenvectors of the covariance matrix. The eigenvectors represent the directions of the principal components, and the eigenvalues show the amount of variance captured by each component.
 - Sort and Select: Sort the eigenvectors by their eigenvalues in descending order and select the top $k$ eigenvectors. These are the directions of the $k$ principal components.

This transformation gives us a low-dimensional representation of the data that retains as much information (variance) as possible.

#### Scikit-learn implementation

In practice, PCA is implemented in `sklearn` as `sklearn.decomposition.PCA` as an object that learns the principal components in its `fit` method and can be used on new data to project it on these components. PCA centers but does not scale the input data for each feature before applying the decomposition. Thus, it is necessary to perform the scaling before sending the data to the PCA algorithm.

## <a name="2">2. Visualization of the Fashion MNIST dataset</a>
(<a href="#0">Go to top</a>)

Fashion-MNIST is a dataset of Zalando's article images, consisting of a data set of 70,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes corresponding to fashion items. Each example is assigned to one of the following labels:

| Label | Description   |
|-------|---------------|
| 0     | T-shirt/top   |
| 1     | Trouser       |
| 2     | Pullover      |
| 3     | Dress         |
| 4     | Coat          |
| 5     | Sandal        |
| 6     | Shirt         |
| 7     | Sneaker       |
| 8     | Bag           |
| 9     | Ankle boot    |

Let's load the dataset and visualize some of its elements. 


In [ ]:
data = fetch_openml(name="Fashion-MNIST")
data.keys()

In [ ]:
# Assemble features and target in same DataFrame for easy data handling
df = data["data"]
# Store target as integer
df["target"] = data["target"].astype(int)
df.head()

Each element in the dataset is an array of 28x28 pixels that can be seen as a vector of 784 features in total, corresponding to the gray scale of the pixel at each location on the 28x28 grid. The class label is an integer ranging from 0 to 9 according to the type of fashion item. 

Let's visualize some of the images from this dataset, separated by classes. You can run the next cell several times, as it picks different random samples from the dataset each time.

In [ ]:
# The 10 classes are 0 to 9 and represent the types of items on the table above
label_description = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot"
}

# Convert class labels to integers for faster lookup
classes = sorted(label_description.keys())

# Group by target and take 5 samples from each class in advance
class_samples = {fashion_class: df[df.target == fashion_class].sample(n=5) for fashion_class in classes}

# Create the figure and axes once
fig, axes = plt.subplots(len(classes), 5, figsize=(10, 18), sharex=True, sharey=True, constrained_layout=True)
fig.suptitle("Example Fashion MNIST images", fontsize=16)

# Plotting the images
for i, fashion_class in enumerate(classes):
    # Set the y-axis label once per row
    axes[i, 0].set_ylabel(f"Class {fashion_class}\n{label_description[fashion_class]}", fontsize=14)

    # Access precomputed samples for the class and plot
    df_class = class_samples[fashion_class]
    for j in range(5):
        image = df_class.iloc[j, :784].astype(float).values.reshape(28, 28)  # Convert and reshape image once
        axes[i, j].imshow(image, cmap="gray_r")

plt.show()

## Dimensionality reduction for data visualization with PCA

The key idea of PCA for dimensionality reduction in data visualization is to transform high-dimensional data into a lower-dimensional space (typically 2D or 3D) while preserving as much of the data's variance (informative content) as possible. This helps us visualize complex datasets by capturing the most important patterns and structures in the data with just a few dimensions.

Let's apply PCA to the Fashion MNIST dataset to represent the images as 2D vectors that can be visualized on the plane. 

We first need to scale the data before applying PCA. We'll apply `numpy`'s `StandardScaler` to standardize the features (pixels in the image) by removing the mean and scaling to unit variance.

In [ ]:
# Scaler to normalize features
scaler = StandardScaler()

# Make a copy of the data so that pop doesn't overwrite df
# This allows this cell to be run multiple times without errors
df_ = df.copy()

# Remove the target as we'll only scale the features
y = df_.pop("target").values
X = df_.values

# Scaled features with zero mean and unit variance
X_sc = scaler.fit_transform(X)

# Shape of the input data
print(f"Shape of features matrix: {X_sc.shape}")

Finally, let's perform PCA on the normalized data. Notice how PCA is an unsupervised ML algorithm that finds patterns on the standardized input features `X_sc` without looking at the labeled classes `y`. 

The number of principal components to retain, given by `n_components`, is a hyperparameter of the algorithm. In this case we reduce the dimensions of the initial images to 2. 

In [ ]:
# Initialize PCA object
pca = PCA(n_components=2)

# Fit PCA to normalized data
X_2D = pca.fit_transform(X_sc)

# Shape of the data after PCA
print(f"Shape of the features matrix after PCA: {X_2D.shape}")

# Print one element of class 0 after the PCA
print(f"\nPCA representation of one element of class 0 (T-shirt/top): {X_2D[y==0][0]}")

As seen above, each of the 70,000 elements of the dataset, originally a matrix of 28x28 = 764 elements, has now been reduced to a 2-dimensional vector, given in the basis of the 2 largest principal components from the PCA. Those 2 directions account for the maximum variance of the original dataset. It is to be expected that some patterns originally present in the high-dimensional data can be observed also in the two-dimensional representation.

Let's plot one of the Fashion MNIST classes, for instance class `0 = Tshirt/top`, and see how the projections on the 2 first principal components look like:

In [ ]:
# Plot the class 0 = T-shirt/top
plt.figure(figsize=(8,6))

# Scatter plot of the fist component [:, 0] against the second [:, 1]
plt.scatter(X_2D[y==0][:, 0], X_2D[y==0][:, 1], s=2, label=label_description[0])
plt.title("2D PCA of Fashion-MNIST dataset", fontsize=14)
plt.xlabel("1st PC", fontsize=12)
plt.ylabel("2nd PC", fontsize=12)
plt.legend()

plt.show()

### Exercise 1

<div style="align: left; border: 4px solid cornflowerblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 80%; max-height:80%; margin: 5px;" src="../../images/MLU_challenge.png" alt="MLU challenge" width=12% height=12%/>
    <span style="padding: 20px; align: left;">
        <p><b>It is your turn!</b></p>
        <p><b>Exercise 1. Visualize class separation of the Fashion MNIST dataset.</b></p>
        <p>Modify the plot above to show more than one class. You can plot all 10 classes, or choose groups of 2, 3 classes to better observe the structure.</p>
        <p>Which classes do you think that will show a clearer separation in the space of the 2 first Principal Components? And which classes do you think that will overlap the most? Check your hypotheses.</p> 
        </span>
</div>

In [ ]:
###### YOUR CODE HERE ######






###### END OF CODE ######

<div style="align: left; border: 4px solid lightcoral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="../../images/MLU_question.png" alt="MLU solution" width=12% height=12%/>
    <span style="padding: 20px; align: left;">
        <p><b>Challenge Help</b></p>
        <p>You can generalize the plotting code above by adding several scatter plots one after the other, or using a for loop.</p>
        <p>If you're stuck, remove the <code>#</code> before the <code>load</code> instruction in the next code cell to display a sample solution.</p>
    </span>
</div>

In [ ]:
# %load solutions/lab22_ex1_solutions.txt

#### Learnings from visualization of high-dimensional data with PCA

The 2-dimensional plots obtained with PCA give an intuitive visual explanation of the structure of the high-dimensional data. Additionally, more dimensions in the construction of the PCA can be considered. For instance one can choose `n_components = 3` and plot the data in a 3-dimensional space. This type of unsupervised techniques are very useful in feature engineering and model selection to decide on the type of ML algorithm to be implemented. For instance, if the PCA shows a clear separation of the classes after reducing the dimensionality, it is likely that a simple linear-based classifier will be able to perform well on a low-dimensional representation of the original data. This can significantly ease the model training process.

## <a name="3">3. Principal Component Regression</a>
(<a href="#0">Go to top</a>)

Principal Component Regression (PCR) combines PCA with linear regression on datasets with labeled numerical targets, i.e. for regression problems rather than classification. It is a way to build a regression model in cases where there are many correlated predictor variables, which can cause issues in regular regression. If we have too many features in our dataset, or if some features are highly correlated, linear regression models can suffer from overfitting and instability. PCR solves this problem by:

 - Reducing the number of features: Using only the top principal components instead of the original features.
 - Improving stability: The principal components are uncorrelated, so they work well in regression models without multicollinearity issues.
 
#### How PCR Works
1. Apply PCA on the predictor variables: First, we apply PCA to the predictor variables (independent variables) to reduce their dimensionality.
2. Select top $k$ components: Choose the top $k$ principal components that capture most of the variance in the predictors.
3. Apply linear regression: Use these $k$ principal components as the new predictors in a linear regression model to predict the target variable.

#### Diabetes data

For this example you will use the `diabetes` dataset, available in `sklearn`. This dataset contains data from diabetic patients, in particular certain features such as their bmi, age, blood pressure, and glucose levels which are useful in predicting the diabetes disease progression in patients. This data can be modeled with a linear regression approach. There expects to be a certain degree of multicollinearity in this data, thus it is worthwhile to explore whether PCA can help reduce the dimensionality of the dataset. This approach is called Principal Component Regression.

Let's start by loading and inspecting the dataset.

In [ ]:
# Load the dataset
diabetes = load_diabetes(as_frame=True)

diabetes["data"].head()

Notice that this data already has its mean removed. Each of the 10 feature variables have been mean centered and scaled by the standard deviation times the square root of the number of sample, i.e. the sum of squares of each column totals 1.

**There's no need to further subtract the mean before sending the data to the PCA for this particular dataset, as it's already centered.**

In [ ]:
# Check that the mean of each column is zero
diabetes["data"].describe()

In [ ]:
# The correlation of each of the features is 1 -- check that values in the diagonal equal 1
(diabetes["data"].T).dot(diabetes["data"])

In [ ]:
# Don't plot the sex data
features = diabetes["feature_names"]
features.remove("sex")

# Plot
fig, axs = plt.subplots(3, 3, figsize=(8, 6))
fig.suptitle("Diabetes Dataset")
for i in range(3):
    for j in range(3):
        n = j + i * 3
        feature = features[n]
        axs[i, j].scatter(diabetes["data"][feature], diabetes["target"], s=1)
        axs[i, j].set_xlabel(feature)
        axs[i, j].set_ylabel("target")
plt.tight_layout()
plt.show()

Let's fit a linear regression using `LinearRegression` as we did in Lab 1 - Ordinary Least Squares. We will compute MSE and $R^2$ for this multivariate regression problem using all 10 features from the dataset.

In [ ]:
# Get features and target from dataframes
X = diabetes["data"].values
y = diabetes["target"].values

# Split data in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=47)

# Train model and print performance metrics
# Use sklearn implementation
lr = LinearRegression()
lr.fit(X_train, y_train)

print(f"sklearn LinearRegression model parameters: {lr.intercept_}, {lr.coef_}")
print(f"Number of sklearn LinearRegression model parameters: {1 + len(lr.coef_)}")
print()
print(f"sklearn LinearRegression training R2: {r2_score(y_train, lr.predict(X_train)):.4f}")
print(f"sklearn LinearRegression test R2: {r2_score(y_test, lr.predict(X_test)):.4f}")
print()
print(f"sklearn LinearRegression training MSE: {mean_squared_error(y_train, lr.predict(X_train)):.4f}")
print(f"sklearn LinearRegression test MSE: {mean_squared_error(y_test, lr.predict(X_test)):.4f}")
print()
print(f"sklearn LinearRegression training RMSE: {root_mean_squared_error(y_train, lr.predict(X_train)):.4f}")
print(f"sklearn LinearRegression test RMSE: {root_mean_squared_error(y_test, lr.predict(X_test)):.4f}")

# Store results for later plot
r2_score_train = r2_score(y_train, lr.predict(X_train))
r2_score_test = r2_score(y_test, lr.predict(X_test))

rmse_train = root_mean_squared_error(y_train, lr.predict(X_train))
rmse_test = root_mean_squared_error(y_test, lr.predict(X_test))

### Exercise 2

<div style="align: left; border: 4px solid cornflowerblue; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 80%; max-height:80%; margin: 5px;" src="../../images/MLU_challenge.png" alt="MLU challenge" width=12% height=12%/>
    <span style="padding: 20px; align: left;">
        <p><b>It is your turn!</b></p>
        <p><b>Exercise 1. Implement Principal Component Regression on the diabetes dataset.</b></p>
        <p>Apply PCA to the data to obtain its principal components. Plot the percentange of the variance explained by each principal component.</p>
        <p>Fit linear regression models to a subset of the transformed features, keeping those with the largest variance.</p> 
        <p>What's an optimal dimensionality for the transformed dataset, that achieves good performance while keeping the dataset small?</p> 
        </span>
</div>

In [ ]:
###### YOUR CODE HERE ######






###### END OF CODE ######

<div style="align: left; border: 4px solid lightcoral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px; width: 65%">
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="../../images/MLU_question.png" alt="MLU solution" width=12% height=12%/>
    <span style="padding: 20px; align: left;">
        <p><b>Challenge Help</b></p>
        <p>You can find the ratios of explained variance per number of chosen principal components in the attribute <code>.explained_variance_ratio</code> of a trained PCA.</p>
        <p>To estimate the optimal number of principal components for the PCR, we recommend fitting linear models to 1, 2, 3, ... principal components and compare standard performance metrics for linear regression, such as RMSE or R2, as a function of the number of chosen components.</p>
        <p>If you're stuck, remove the <code>#</code> before the <code>load</code> instruction in the next code cell to display a sample solution.</p>
    </span>
</div>

In [ ]:
# %load solutions/lab22_ex2_solutions.txt

<div style="display: flex; align-items: center; justify-content: left; background-color:#330066; width:99%;"> 
        <img style="float: left; max-width: 100%; max-height:100%; margin: 15px;" src="../../images/MLU_robot.png" alt="MLU robot" width="100" height="100"/>
    <span style="color: white; padding-left: 10px; align: left; margin: 15px;">
        <h3>Congratulations!</h3>
        You have completed Lab 2.2: Dimensionality reduction and Principal Component Regression of Lecture 2: Advanced linear algebra of MLU Mathematical Fundamentals of Machine Learning.
        <br/>
    </span>
</div>